# CLASE 15 - Análisis de Datos con Python
#### Comisión 25261 - Palazzesi / Sosa

## Ejercicio práctico:

1. Se solicita la creación de un dashboard que muestre la cantidad de seguidores, "me gusta" y comentarios en las redes sociales de la empresa. Es importante poder visualizar la evolución de estas métricas a lo largo del tiempo y filtrar por plataforma para identificar áreas de mayor impacto.

2. Desarrollo: Cargar el dataset de redes sociales. Utilizar Dash y Plotly para crear un dashboard con los siguientes elementos:

  * Un gráfico de líneas que muestre la evolución de la cantidad de seguidores a lo largo del tiempo para cada plataforma (Facebook, Twitter, Instagram, etc.).

  * Un gráfico de barras que compare la cantidad de "me gusta" y comentarios en cada plataforma. ●​ Filtros que permitan a los usuarios seleccionar una o varias plataformas y un período de tiempo específico.
  
  * Títulos claros y concisos para cada gráfico.

## Carga y preparación de los datos

In [6]:
# ----------------------------------------------------
# Importamos modulos necesarios
# ----------------------------------------------------
import pandas as pd
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go

# ----------------------------------------------------
# Cargar el dataset
# ----------------------------------------------------
ruta_csv = "rrss.csv"
df = pd.read_csv(ruta_csv)

# ----------------------------------------------------
# Asegurarnos de que la columna fecha sea de tipo datetime
# ----------------------------------------------------
df["fecha"] = pd.to_datetime(df["fecha"])

# ----------------------------------------------------
# Vistazo rapido
# ----------------------------------------------------
display(df.head())
print()
print(df.dtypes)
print()

# ----------------------------------------------------
# Guardamos info util para los filtros
# ----------------------------------------------------
fecha_min = df["fecha"].min()
fecha_max = df["fecha"].max()
plataformas = sorted(df["plataforma"].unique())
print()
print("Rango de fechas:", fecha_min.date(), "->", fecha_max.date())
print()
print("Plataformas:", plataformas)


,fecha,plataforma,seguidores,likes,comentarios
0,2024-07-09,TikTok,798,88,26
1,2025-02-15,YouTube,1300,113,7
2,2025-04-22,Facebook,1392,108,25
3,2025-05-24,X,828,133,11
4,2025-01-24,Facebook,847,98,26



fecha          datetime64[ns]
plataforma             object
seguidores              int64
likes                   int64
comentarios             int64
dtype: object


Rango de fechas: 2024-06-09 -> 2025-06-08

Plataformas: ['Facebook', 'Instagram', 'TikTok', 'X', 'YouTube']


## Dashboard con Dash + Plotly

In [7]:
# ----------------------------------------------------
# Graficos individuales a incluir en el dashboard
# ----------------------------------------------------

# ----------------------------------------------------
# 1) Lineas: evolucion de seguidores en el tiempo
# por plataforma
# ----------------------------------------------------
df_line = df.sort_values("fecha")

fig_line = px.line(
    df_line,
    x="fecha",
    y="seguidores",
    color="plataforma",
    markers=True,
    title="Evolucion de seguidores por plataforma"
)

fig_line.update_layout(
    xaxis_title="Fecha",
    yaxis_title="Cantidad de seguidores"
)

# Agregamos un range slider para filtrar periodo de tiempo
fig_line.update_xaxes(rangeslider_visible=True)


# ----------------------------------------------------
# 2) Barras: likes totales por plataforma
# ----------------------------------------------------
df_likes = df.groupby("plataforma", as_index=False)["likes"].sum()

fig_bar_likes = px.bar(
    df_likes.sort_values("likes", ascending=False),
    x="plataforma",
    y="likes",
    title="Likes totales por plataforma",
    text_auto=True
)

fig_bar_likes.update_layout(
    xaxis_title="Plataforma",
    yaxis_title="Likes totales"
)

# ----------------------------------------------------
# 3) Barras agrupadas: likes vs comentarios por
# plataforma
# ----------------------------------------------------

df_likes_com = df.groupby("plataforma", as_index=False)[["likes", "comentarios"]].sum()

df_melt = df_likes_com.melt(
    id_vars="plataforma",
    value_vars=["likes", "comentarios"],
    var_name="metrica",
    value_name="cantidad"
)

fig_bar_lc = px.bar(
    df_melt,
    x="plataforma",
    y="cantidad",
    color="metrica",
    barmode="group",
    title="Likes vs comentarios por plataforma",
    text_auto=True
)

fig_bar_lc.update_layout(
    xaxis_title="Plataforma",
    yaxis_title="Cantidad"
)


# ----------------------------------------------------
# Creamos la figura del tablero con subplots
# ----------------------------------------------------
fig_dashboard = make_subplots(
    rows=2, cols=2,
    specs=[
        [{"type": "xy"}, {"type": "xy"}],
        [{"type": "xy", "colspan": 2}, None]
    ],
    subplot_titles=(
        "Seguidores en el tiempo por plataforma",
        "Likes totales por plataforma",
        "Likes vs comentarios por plataforma"
    )
)

# ----------------------------------------------------
# Agregamos los graficos en cada "hueco" del dashboard
# ----------------------------------------------------

# Lineas (fila 1, col 1)
for trace in fig_line.data:
    fig_dashboard.add_trace(trace, row=1, col=1)

# Barras de likes (fila 1, col 2)
for trace in fig_bar_likes.data:
    fig_dashboard.add_trace(trace, row=1, col=2)

# Barras likes vs comentarios (fila 2, col 1-2)
for trace in fig_bar_lc.data:
    fig_dashboard.add_trace(trace, row=2, col=1)


# ----------------------------------------------------
# Ajustes de ejes y layout general
# ----------------------------------------------------
# Subplot 1,1 (lineas)
fig_dashboard.update_xaxes(title_text="Fecha", row=1, col=1)
fig_dashboard.update_yaxes(title_text="Seguidores", row=1, col=1)

# Activamos range slider en el subplot 1,1
fig_dashboard.update_xaxes(rangeslider_visible=True, row=1, col=1)

# Subplot 1,2 (likes por plataforma)
fig_dashboard.update_xaxes(title_text="Plataforma", row=1, col=2)
fig_dashboard.update_yaxes(title_text="Likes totales", row=1, col=2)

# Subplot 2,1 (likes vs comentarios)
fig_dashboard.update_xaxes(title_text="Plataforma", row=2, col=1)
fig_dashboard.update_yaxes(title_text="Cantidad", row=2, col=1)

fig_dashboard.update_layout(
    height=700,
    width=1000,
    title_text="Dashboard de redes sociales (Plotly interactivo)",
    showlegend=True,
    margin=dict(l=40, r=40, t=80, b=40)
)

# ----------------------------------------------------
# Mostramos el dashboard
# ----------------------------------------------------
fig_dashboard.show()


### Explicación del código anterior:

Claro, te lo explico de manera fluida y sin usar listas.

La parte del código que crea el dashboard funciona combinando varias figuras interactivas dentro de una única figura general. Primero se preparan los gráficos individuales con Plotly Express: uno de líneas para ver cómo evoluciona la cantidad de seguidores en el tiempo y dos gráficos de barras que muestran las interacciones por plataforma. Cada uno de esos gráficos queda guardado en un objeto diferente, como `fig_line`, `fig_bar_likes` y `fig_bar_lc`.

Luego se construye la estructura del tablero con `make_subplots`. Esta función define un lienzo general que tendrá dos filas y dos columnas, aunque la segunda fila usa ambas columnas como un solo espacio grande. Esa estructura queda vacía al principio; es simplemente un contenedor donde después se irán insertando los gráficos ya preparados.

Cada gráfico de Plotly está formado por uno o más "traces". Un trace es una serie gráfica: por ejemplo, una línea para Facebook, otra para Instagram y otra para Twitter. Cuando se quiere llevar uno de esos gráficos a un subplot, se recorre la lista `fig_line.data` y se agrega cada trace al lugar que corresponde del tablero con `add_trace`, especificando fila y columna.

Después de colocar todos los gráficos en sus posiciones dentro del dashboard, se ajustan manualmente algunos aspectos para que todo quede bien presentado. Se renombra cada eje según lo que debería mostrar, se activa el rango deslizante en el eje de fechas del primer gráfico para poder hacer zoom temporal, y se ajusta el tamaño total del tablero para que quede cómodo de ver en pantalla.

Finalmente, se muestra la figura completa con `fig_dashboard.show()`. En ese momento, Plotly combina todas las piezas: los tres gráficos independientes, la estructura del tablero, los subtítulos y la configuración final del diseño. El resultado es un único dashboard interactivo donde los gráficos conviven y se relacionan visualmente como si fueran parte de una misma interfaz.
